## Training Langue

In [7]:


import torch
import os
from samgeo.text_sam import LangSAM
# sam_vit_h_4b8939
# sam_vit_l_0b3195
# sam_vit_b_01ec64

chkpnt_dir = r"/home/skm/SKM/WORK/ALL_CODE/WorkSpace_SKM_DucAnh/SAM/model/"
checkpoint = os.path.join(chkpnt_dir, "sam_vit_b_01ec64.pth")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = LangSAM()

text_prompt = "tree"
image = r"/home/skm/SKM/WORK/ALL_CODE/WorkSpace_SKM_DucAnh/SAM/data_demo/a_4326.tif"
sam.predict(image, text_prompt, box_threshold=0.24, text_threshold=0.24)

NameError: name 'SLConfig' is not defined

In [3]:
import geopandas as gpd
a = r'/home/skm/public_mount/tmp_Duy/point_sam_2/point_sam_2.shp'
df_prompt = gpd.read_file(a)


In [16]:
a = df_prompt['id'].to_list()
a

[1, 0, 1, 0, 1, 0, 1, 0]

In [20]:
points = df_prompt['geometry'].to_list()
points

[<POINT (-10616527.07 3462156.134)>,
 <POINT (-10616528.569 3462147.261)>,
 <POINT (-10616528.569 3462137.649)>,
 <POINT (-10616526 3462128.899)>,
 <POINT (-10616528.355 3462122.614)>,
 <POINT (-10616512.723 3462133.089)>,
 <POINT (-10616540.508 3462122.552)>,
 <POINT (-10616537.617 3462128.221)>]

In [59]:
point_shape = r'/home/skm/SKM/WORK/ALL_CODE/WorkSpace_SKM_DucAnh/SAM/data_demo/a_4326.shp'
a = r'/home/skm/public_mount/tmp_Duy/point_sam_2/point_sam_2.shp'


gdf = gpd.read_file(a)
if gdf.geom_type.unique()[0] == 'Point':
    print("Shapefile has Point geometry")

# Check if 'id' field exists in the shapefile
if 'id' in gdf.columns:
    print("Shapefile has 'id' field")



Shapefile has Point geometry
Shapefile has 'id' field


In [52]:
geometry_list = []
geometry_list = gdf['geometry']

# Convert each geometry element to a pair of lists
pair_list = []
for geometry in geometry_list:
    pair_list.append([geometry.coords[0]])
pair_list

[[(-10616527.070439188, 3462156.1341076526)],
 [(-10616528.569428362, 3462147.261122024)],
 [(-10616528.569428362, 3462137.6487278226)],
 [(-10616525.999732636, 3462128.8989906185)],
 [(-10616528.355287053, 3462122.613971788)],
 [(-10616512.722971382, 3462133.0890048766)],
 [(-10616540.507806422, 3462122.5523539707)],
 [(-10616537.61689873, 3462128.2211943227)]]

In [58]:
epsg_code = "EPSG:" + str(gdf.crs.to_epsg())
epsg_code

'EPSG:3857'

In [64]:
import os
import torch
from samgeo import SamGeo
import geopandas as gpd

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def run_sam_point(model_path, img_path, input_prompt_shp, outputVector, dir_tmp):
    # check shp point
    df_prompt = gpd.read_file(input_prompt_shp)
    if df_prompt.geom_type.unique()[0] == 'Point' and 'id' in df_prompt.columns:
        os.makedirs(os.path.dirname(outputVector), exist_ok=True)
        os.makedirs(dir_tmp, exist_ok=True)
        
        sam = SamGeo(
                checkpoint=model_path,
                model_type="vit_h", # vit_l, vit_b , vit_h
                automatic=False,
                device=device,
                sam_kwargs=None,
            )
        sam.set_image(img_path)
        df_point = df_prompt['geometry']
        # Convert each geometry element to a pair of lists
        point_lists = []
        for geometry in df_point:
            point_lists.append([geometry.coords[0]])
        point_labels = df_prompt['id'].to_list()
        
        print(len(point_labels))
        print(len(point_labels), point_labels)
        point_labels = point_labels[1]
        point_lists = point_lists[1]
        
        epsg_code = "EPSG:" + str(df_prompt.crs.to_epsg())
        outputRaster_tmp = os.path.join(dir_tmp, os.path.basename(img_path))
        sam.predict(point_lists, point_labels=point_labels, point_crs=epsg_code, output=outputRaster_tmp)
        sam.raster_to_vector(outputRaster_tmp, outputVector, simplify_tolerance=0.00001)
    else:
        print('shapefile dont have "id" field or Point geometry')
  
model_path = r"/home/skm/SKM/WORK/ALL_CODE/WorkSpace_SKM_DucAnh/SAM/model/sam_vit_h_4b8939.pth"
img_path = r'/home/skm/SKM/WORK/ALL_CODE/WorkSpace_SKM_DucAnh/SAM/data_demo/a_4326.tif'
input_prompt_shp = r'/home/skm/public_mount/tmp_Duy/point_sam_2/point_sam_2.shp'
outputVector = r'/home/skm/SKM/WORK/ALL_CODE/WorkSpace_SKM_DucAnh/SAM/data_demo/TEST_DOCKER/meo.shp'
dir_tmp = r'/home/skm/SKM/WORK/ALL_CODE/WorkSpace_SKM_DucAnh/SAM/data_demo/TEST_DOCKER/tmp'

run_sam_point(model_path, img_path, input_prompt_shp, outputVector, dir_tmp)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.92 GiB total capacity; 9.77 GiB already allocated; 20.50 MiB free; 9.88 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [14]:
a = 5000  # Số a muốn tìm
b = 200  # Số b muốn tìm
min_x = 1000
max_x = 1500

found_x = None

for x in range(min_x, max_x + 1):
    if a % x > 100 and b % x > 100 and b % x < x:
        found_x = x
        break

if found_x is not None:
    print("Tìm thấy x =", found_x)
else:
    print("Không tìm thấy x thỏa mãn yêu cầu.")


Tìm thấy x = 1001


In [15]:
5000%1001

996

In [17]:
a = 1
if a:
    print(1)

1


In [26]:
import numpy as np

original_array = np.random.rand(3, 255, 1111)  # Mảng numpy ban đầu

# Giảm số chiều của mảng xuống còn một chiều
new_array = original_array.reshape(1, 255, 1111)
new_array.shape

ValueError: cannot reshape array of size 849915 into shape (1,255,1111)

In [29]:
zeros_array = np.zeros((1,255, 222))
zeros_array.shape

(1, 255, 222)